## **-- LISTE DES ELEVES PROPRE ET COMPLETE --**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import email
from email import policy
from email.parser import BytesParser
import re

In [2]:
eleve = pd.read_csv(r'C:\Users\steph\Documents\projet\liste_eleve.csv', encoding='ISO-8859-1', delimiter=';', header=0)

In [3]:
# Afficher les 5 premières lignes du DataFrame
print(eleve.head())

# Afficher des informations sur le DataFrame
print(eleve.info())

   NÂ°          Nom   PrÃ©nom   NÃ©(e) le Sexe Niveau
0    1     AUGEREAU  Amandine  09/04/2016    F     PS
1    2  BLEVIN KALT      Malo  15/08/2016    M     PS
2    3     BOUFFARD      Jade  27/03/2016    F     PS
3    4      BOURGES    Thï¿½o  28/07/2016    M     PS
4    5         COTO     Louis  13/04/2016    M     PS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   NÂ°        24 non-null     int64 
 1   Nom        24 non-null     object
 2   PrÃ©nom    24 non-null     object
 3   NÃ©(e) le  24 non-null     object
 4   Sexe       24 non-null     object
 5   Niveau     24 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.3+ KB
None


In [4]:
# Affichez les noms de colonnes pour vérifier le bon nom
print(eleve.columns)

# Remplacez les caractères spéciaux
eleve['PrÃ©nom'] = eleve['PrÃ©nom'].str.replace('ï¿½', 'e')
print(eleve)

Index(['NÂ°', 'Nom', 'PrÃ©nom', 'NÃ©(e) le', 'Sexe', 'Niveau'], dtype='object')
    NÂ°            Nom    PrÃ©nom   NÃ©(e) le Sexe Niveau
0     1       AUGEREAU   Amandine  09/04/2016    F     PS
1     2    BLEVIN KALT       Malo  15/08/2016    M     PS
2     3       BOUFFARD       Jade  27/03/2016    F     PS
3     4        BOURGES       Theo  28/07/2016    M     PS
4     5           COTO      Louis  13/04/2016    M     PS
5     6        DERONNE       Theo  12/02/2016    M     PS
6     7        EHRHART    Georges  29/05/2016    M     PS
7     8        ETCHART       Leon  21/01/2016    M     PS
8     9         FLAMEN     Victor  27/03/2016    M     PS
9    10   FOTSO LAUDEN      Clara  29/07/2016    F     PS
10   11          JEMNI     Kinane  14/03/2016    M     PS
11   12         LEDOUX      Brune  16/12/2016    F     PS
12   13       PERCEROU      Diane  29/04/2016    F     PS
13   14          REDON   Apolline  14/01/2016    F     PS
14   15         SEVENO   Eleonore  18/01/2016    F

In [5]:
chemin_fichier_eml = r'C:\Users\steph\Documents\projet\histoire.eml'

PRENOM = 'PrÃ©nom'
NIVEAU = 'Niveau'
ADRESSE_EMAIL = 'Adresse e-mail'

def ouvrir_fichier(chemin_fichier):
    try:
        with open(chemin_fichier, 'rb') as fichier_eml:
            return BytesParser(policy=policy.default).parse(fichier_eml)
    except FileNotFoundError:
        print(f"Le fichier {chemin_fichier} n'a pas été trouvé.")
        return None

def extraire_destinataires(message):
    destinataires = []
    for destinataire in message.get_all('to', []):
        for nom, adresse_mail in email.utils.getaddresses([destinataire]):
            nom = nom.replace('"', '').title()
            nom = nom.replace('Apoliine', 'Apolline').title()
            nom = nom.replace('Zoé', 'Zoe').title()
            nom = nom.replace('Inès', 'Ines').title()
            niveau = None
            if "Ps" in nom:
                niveau = "PS"
                nom = nom.replace("Ps", "")
            elif "Ms" in nom:
                niveau = "MS"
                nom = nom.replace("Ms", "")
            prenom = nom.split()[0] if nom else None
            destinataires.append({PRENOM: prenom, NIVEAU: niveau, ADRESSE_EMAIL: adresse_mail})
    return destinataires

def fusionner_dataframes(df1, df2):
    df1[PRENOM] = df1[PRENOM].str.title()
    df2[PRENOM] = df2[PRENOM].str.title()
    df = pd.merge(df1, df2, how='left', on=PRENOM, suffixes=('_eleve', '_dest'))
    df = df.drop_duplicates()
    return df

# Utilisation des fonctions
message = ouvrir_fichier(chemin_fichier_eml)
if message:
    destinataires = extraire_destinataires(message)
    destinataires_df = pd.DataFrame(destinataires)
    eleve = fusionner_dataframes(eleve, destinataires_df)
print(eleve[['Nom', 'PrÃ©nom', 'Niveau_eleve', 'Adresse e-mail']])

              Nom    PrÃ©nom Niveau_eleve                 Adresse e-mail
0        AUGEREAU   Amandine           PS           miliefou@hotmail.com
1     BLEVIN KALT       Malo           PS              agneskalt@msn.com
2        BOUFFARD       Jade           PS         bouffard.jul@gmail.com
3         BOURGES       Theo           PS      chirio.jennifer@gmail.com
4            COTO      Louis           PS        clairerobert4@gmail.com
5         DERONNE       Theo           PS      chirio.jennifer@gmail.com
6         EHRHART    Georges           PS      berengere.hardy@gmail.com
7         ETCHART       Leon           PS          evi.vega84@icloud.com
8          FLAMEN     Victor           PS            gerthies@hotmail.fr
9    FOTSO LAUDEN      Clara           PS          elise.lauden@yahoo.fr
10          JEMNI     Kinane           PS      kilanihinda33@hotmail.com
11         LEDOUX      Brune           PS     elodielelievre@hotmail.com
12       PERCEROU      Diane           PS          

In [6]:
print(eleve.columns)

Index(['NÂ°', 'Nom', 'PrÃ©nom', 'NÃ©(e) le', 'Sexe', 'Niveau_eleve',
       'Niveau_dest', 'Adresse e-mail'],
      dtype='object')
